In [1]:
import os
import sys

PROJECT_ROOT = os.path.abspath(os.path.join(
                  os.path.dirname("test-pretrained"), 
                  os.pardir)
)
sys.path.append(PROJECT_ROOT)

import pandas as pd
import numpy as np
import torch
import json
import random
import torch.nn as nn
import torch.nn.functional as F
from copy import deepcopy
from src.data_augmentation import Mixup, Specmix, Cutmix
from src.features import extract_wavelet_from_raw_audio
from src.dataset import create_dataloader
from src.utils import feature_extraction_pipeline, read_features_files, choose_model, read_feature, pad_features
from src.models.utils import SaveBestModel, weight_init
from src.models.cnn3 import *
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
from typing import Dict, Tuple, List, Union, Iterable
from sklearn.metrics import classification_report

# making sure the experiments are reproducible
seed = 2109
random.seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

def seed_worker(worker_id: int):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(seed)

In [2]:
def train(
    model: nn.Module,
    dataloader: DataLoader,
    optimizer: torch.optim.Adam,
    loss: torch.nn.CrossEntropyLoss,
    device: torch.device,
    mixer: Union[None, Mixup, Specmix, Cutmix]
) -> Tuple[float, float]:
    """
    Function responsible for the model training.

    Args:
        model (nn.Module): the created model.
        dataloader (DataLoader): the training dataloader.
        optimizer (torch.optim.Adam): the optimizer used.
        loss (torch.nn.CrossEntropyLoss): the loss function used.
        device (torch.device): which device to use.

    Returns:
        Tuple[float, float]: the training f1 and loss, respectively.
    """
    model.train()
    predictions = []
    targets = []
    train_loss = 0.0
    
    for index, (batch) in enumerate(dataloader, start=1):
        data = batch["features"].to(device)
        target = batch["labels"].to(device)
        optimizer.zero_grad()
        
        data = data.to(dtype=torch.float32)
        target = target.to(dtype=torch.float32)
        
        if not mixer is None:
            data, target = mixer(
                x=data,
                y=target
            )
            
        output = model(data)

        l = loss(output, target)
        train_loss += l.item()
        
        l.backward()
        optimizer.step()
        
        prediction = output.argmax(dim=-1, keepdim=True).to(dtype=torch.int)
        prediction = prediction.detach().cpu().numpy()
        predictions.extend(prediction.tolist())
        
        target = target.argmax(dim=-1, keepdim=True).to(dtype=torch.int)
        target = target.detach().cpu().numpy()
        targets.extend(target.tolist())
        
    train_loss = train_loss/index
    train_f1 = classification_report(
        targets,
        predictions,
        digits=6,
        output_dict=True,
        zero_division=0.0
    )
    train_f1 = train_f1["macro avg"]["f1-score"]
    return train_f1, train_loss

def evaluate(
    model: nn.Module,
    dataloader: DataLoader,
    loss: torch.nn.CrossEntropyLoss,
    device: torch.device
) -> Tuple[float, float]:
    """
    Function responsible for the model evaluation.

    Args:
        model (nn.Module): the created model.
        dataloader (DataLoader): the validaiton dataloader.
        loss (torch.nn.CrossEntropyLoss): the loss function used.
        device (torch.device): which device to use.

    Returns:
        Tuple[float, float]: the validation f1 and loss, respectively.
    """
    model.eval()
    predictions = []
    targets = []
    validation_loss = 0.0
    validation_f1 = []
    
    with torch.inference_mode():
        for index, (batch) in enumerate(dataloader):
            data = batch["features"].to(device)
            target = batch["labels"].to(device)

            data = data.to(dtype=torch.float32)
            target = target.to(dtype=torch.float32)
                        
            output = model(data)
            
            l = loss(output, target)
            validation_loss += l.item()
            
            prediction = output.argmax(dim=-1, keepdim=True).to(dtype=torch.int)
            prediction = prediction.detach().cpu().numpy()
            predictions.extend(prediction.tolist())
            
            target = target.argmax(dim=-1, keepdim=True).to(dtype=torch.int)
            target = target.detach().cpu().numpy()
            targets.extend(target.tolist())
    
    validation_loss = validation_loss/index
    validation_f1 = classification_report(
        targets,
        predictions,
        digits=6,
        output_dict=True,
        zero_division=0.0
    )
    validation_f1 = validation_f1["macro avg"]["f1-score"]
    return validation_f1, validation_loss

def test(
    model: nn.Module,
    dataloader: DataLoader,
    device: torch.device
) -> str:
    """
    Function responsible for the model testing in the test dataset.

    Args:
        model (nn.Module): the created model.
        dataloader (DataLoader): the test dataloader.
        device (torch.device): which device to use.

    Returns:
        str: the test classification report.
    """
    model.eval()
    predictions = []
    targets = []
    
    with torch.inference_mode():
        for batch in dataloader:
            data = batch["features"].to(device)
            target = batch["labels"].to(device)

            data = data.to(dtype=torch.float32)
            target = target.to(dtype=torch.float32)
            
            output = model(data)
            
            prediction = output.argmax(dim=-1, keepdim=True).to(dtype=torch.int)
            prediction = prediction.detach().cpu().numpy()
            predictions.extend(prediction.tolist())
            
            target = target.argmax(dim=-1, keepdim=True).to(dtype=torch.int)
            target = target.detach().cpu().numpy()
            targets.extend(target.tolist())
    
    class_report = classification_report(
        targets,
        predictions,
        digits=4,
        output_dict=True
    )
    return class_report

In [3]:
features_path = "../features8k/propor2022/"

# loading training features
X_train = read_feature(path=features_path, fold="0", name="X_train.pth")
y_train = read_feature(path=features_path, fold="0", name="y_train.pth")
print(f"Train: {X_train.shape}, {y_train.shape}")

# loading validation features
X_valid = read_feature(path=features_path, fold="0", name="X_valid.pth")
y_valid = read_feature(path=features_path, fold="0", name="y_valid.pth")
print(f"Valid: {X_valid.shape}, {y_valid.shape}")

# loading testing features
X_test = read_feature(path=features_path, fold=None, name="X_test.pth")
y_test = read_feature(path=features_path, fold=None, name="y_test.pth")
print(f"Test: {X_test.shape}, {y_test.shape}")

Train: torch.Size([500, 1, 128000]), torch.Size([500, 3])
Valid: torch.Size([125, 1, 128000]), torch.Size([125, 3])
Test: torch.Size([308, 1, 128000]), torch.Size([308, 3])


In [6]:
# reading the parameters configuration file
params = json.load(open("../config/mode_2.json", "r"))

feature_config = params["feature"]
feature_config["sample_rate"] = int(params["sample_rate"])
data_augmentation_config = params["data_augmentation"]
dataset = params["dataset"]
wavelet_config = params["wavelet"]
mode = params["mode"]

model_config = params["model"]
model_config["name"] = "cnn2"
model_config["use_gpu"] = False
model_config["batch_size"] = 16
device = torch.device("cuda" if torch.cuda.is_available and model_config["use_gpu"] else "cpu")

if dataset == "propor2022":
    if data_augmentation_config["target"] == "majority":
        data_augment_target = [0]
    elif data_augmentation_config["target"] == "minority":
        data_augment_target = [1, 2]
    elif data_augmentation_config["target"] == "all":
        data_augment_target = [0, 1, 2]
    else:
        raise ValueError("Invalid arguments for target. Should be 'all', 'majority' or 'minority")
else:
    raise NotImplementedError
    
model = choose_model(
    mode=mode,
    model_name=model_config["name"],
    device=device
)

In [7]:
optimizer = torch.optim.Adam(
    params=model.parameters(),
    lr=model_config["learning_rate"],
    betas=(0.9, 0.98),
    eps=1e-9
)
loss = torch.nn.CrossEntropyLoss()
scheduler = None
mixer = None

# creating the model checkpoint object
sbm = SaveBestModel(
    output_dir=os.path.join(model_config["output_path"], dataset, mode, model_config["name"]),
    model_name=model_config["name"]
)

if model_config["use_lr_scheduler"]:
    print("\nWARNING: Using learning rate scheduler!\n")
    scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

if "mixup" in data_augmentation_config["techniques"].keys():
    print("\nWARNING: Using mixup data augmentation technique!\n")
    mixer = Mixup(
        alpha=data_augmentation_config["techniques"]["mixup"]["alpha"]
    )

if "specmix" in data_augmentation_config["techniques"].keys():
    print("\nWARNING: Using specmix data augmentation technique!\n")
    mixer = Specmix(
        p=data_augmentation_config["p"],
        min_band_size=data_augmentation_config["techniques"]["specmix"]["min_band_size"],
        max_band_size=data_augmentation_config["techniques"]["specmix"]["max_band_size"],
        max_frequency_bands=data_augmentation_config["techniques"]["specmix"]["max_frequency_bands"],
        max_time_bands=data_augmentation_config["techniques"]["specmix"]["max_time_bands"],
        device=device
    )

if "cutmix" in data_augmentation_config["techniques"].keys():
    print("\nWARNING: Using cutmix data augmentation technique!\n")
    mixer = Cutmix(
        alpha=data_augmentation_config["techniques"]["cutmix"]["alpha"],
        p=data_augmentation_config["p"]
    )

# creating the training dataloader
training_dataloader = create_dataloader(
    X=X_train,
    y=y_train,
    feature_config=feature_config,
    wavelet_config=wavelet_config,
    data_augmentation_config=data_augmentation_config,
    num_workers=0,
    mode=mode,
    shuffle=False,
    training=True,
    batch_size=model_config["batch_size"],
    data_augment_target=data_augment_target,
    worker_init_fn=seed_worker,
    generator=g
)

# creating the validation dataloader
validation_dataloader = create_dataloader(
    X=X_valid,
    y=y_valid,
    feature_config=feature_config,
    wavelet_config=wavelet_config,
    data_augmentation_config=data_augmentation_config,
    num_workers=0,
    mode=mode,
    shuffle=False,
    training=False,
    batch_size=model_config["batch_size"],
    data_augment_target=data_augment_target,
    worker_init_fn=seed_worker,
    generator=g
)

# creating the test dataloader
test_dataloader = create_dataloader(
    X=X_test,
    y=y_test,
    feature_config=feature_config,
    wavelet_config=wavelet_config,
    data_augmentation_config=None,
    num_workers=0,
    mode=params["mode"],
    shuffle=False,
    training=False,
    batch_size=params["model"]["batch_size"],
    data_augment_target=None,
    worker_init_fn=seed_worker,
    generator=g
)
    
# training loop
for epoch in range(1, model_config["epochs"] + 1):
    print(f"Epoch: {epoch}/{model_config['epochs']}")

    train_f1, train_loss = train(
        device=device,
        dataloader=training_dataloader,
        optimizer=optimizer,
        model=model,
        loss=loss,
        mixer=mixer
    )

    valid_f1, valid_loss = evaluate(
        device=device,
        dataloader=validation_dataloader,
        model=model,
        loss=loss
    )
    
    report = test(
        model=model,
        dataloader=test_dataloader,
        device=device
    )

    test_f1 = report["macro avg"]["f1-score"]

    print(f"\nEpoch: {epoch}")
    print(f"Train F1-Score: {train_f1:1.6f}")
    print(f"Train Loss: {train_loss:1.6f}")
    print(f"Validation F1-Score: {valid_f1:1.6f}")
    print(f"Validation Loss: {valid_loss:1.6f}")
    print(f"Test F1-Score: {test_f1:1.6f}\n")

    # updating learning rate
    if not scheduler is None:
        scheduler.step()

Epoch: 1/100


/media/greca/HD/anaconda3/envs/ser/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/greca/HD/anaconda3/envs/ser/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/greca/HD/anaconda3/envs/ser/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu


Epoch: 1
Train F1-Score: 0.316996
Train Loss: 1.053398
Validation F1-Score: 0.294643
Validation Loss: 1.135530
Test F1-Score: 0.297362

Epoch: 2/100


/media/greca/HD/anaconda3/envs/ser/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/greca/HD/anaconda3/envs/ser/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/greca/HD/anaconda3/envs/ser/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu


Epoch: 2
Train F1-Score: 0.292975
Train Loss: 0.952812
Validation F1-Score: 0.294643
Validation Loss: 1.048462
Test F1-Score: 0.297362

Epoch: 3/100


...

/media/greca/HD/anaconda3/envs/ser/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/greca/HD/anaconda3/envs/ser/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/greca/HD/anaconda3/envs/ser/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu


Epoch: 73
Train F1-Score: 0.292975
Train Loss: 0.644721
Validation F1-Score: 0.294643
Validation Loss: 0.743287
Test F1-Score: 0.297362

Epoch: 74/100


/media/greca/HD/anaconda3/envs/ser/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/greca/HD/anaconda3/envs/ser/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/greca/HD/anaconda3/envs/ser/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu


Epoch: 74
Train F1-Score: 0.292975
Train Loss: 0.643833
Validation F1-Score: 0.294643
Validation Loss: 0.744556
Test F1-Score: 0.297362

Epoch: 75/100


KeyboardInterrupt: 